In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
train=pd.read_csv('/content/drive/MyDrive/Ltassign1datafiles/train_data.csv')

In [ ]:
print(train.head())

   case_id  Hospital_code Hospital_type_code  ...    Age Admission_Deposit   Stay
0        1              8                  c  ...  51-60            4911.0   0-10
1        2              2                  c  ...  51-60            5954.0  41-50
2        3             10                  e  ...  51-60            4745.0  31-40
3        4             26                  b  ...  51-60            7272.0  41-50
4        5             26                  b  ...  51-60            5558.0  41-50

[5 rows x 18 columns]


In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Ltassign1datafiles/test_data.csv')

In [ ]:
print(test.head())

   case_id  Hospital_code  ...    Age  Admission_Deposit
0   318439             21  ...  71-80             3095.0
1   318440             29  ...  71-80             4018.0
2   318441             26  ...  71-80             4492.0
3   318442              6  ...  71-80             4173.0
4   318443             28  ...  71-80             4161.0

[5 rows x 17 columns]


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Number of Null Values
display(train.isna().sum())
display(test.isna().sum())

# Unique Values in Each Column
display(train.nunique())
display(test.nunique())

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
patientid                               0
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

case_id                              318438
Hospital_code                            32
Hospital_type_code                        7
City_Code_Hospital                       11
Hospital_region_code                      3
Available Extra Rooms in Hospital        18
Department                                5
Ward_Type                                 6
Ward_Facility_Code                        6
Bed Grade                                 4
patientid                             92017
City_Code_Patient                        37
Type of Admission                         3
Severity of Illness                       3
Visitors with Patient                    28
Age                                      10
Admission_Deposit                      7300
Stay                                     11
dtype: int64

case_id                              137057
Hospital_code                            32
Hospital_type_code                        7
City_Code_Hospital                       11
Hospital_region_code                      3
Available Extra Rooms in Hospital        15
Department                                5
Ward_Type                                 6
Ward_Facility_Code                        6
Bed Grade                                 4
patientid                             39607
City_Code_Patient                        37
Type of Admission                         3
Severity of Illness                       3
Visitors with Patient                    27
Age                                      10
Admission_Deposit                      6609
dtype: int64

In [ ]:
train.shape,test.shape

((318438, 18), (137057, 17))

In [ ]:
train=train.drop('case_id',axis=1)
test=test.drop('case_id',axis=1)

In [ ]:
train.shape,test.shape

((318438, 17), (137057, 16))

In [ ]:
# Selecting Categorical Columns for Label Encoding
cat_train=train.select_dtypes(include=['object'])
cat_col=list(cat_train.columns)
cat_col_test=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']

In [ ]:
# Label encoding
le=LabelEncoder()
for cat_column in cat_col_test:
    train[cat_column]=le.fit_transform(train[cat_column])
    test[cat_column]=le.transform(test[cat_column])
train['Stay']=le.fit_transform(train['Stay'])
print(le.inverse_transform([0,1,2,3,4,5,6,7,8,9,10,]))

['0-10' '11-20' '21-30' '31-40' '41-50' '51-60' '61-70' '71-80' '81-90'
 '91-100' 'More than 100 Days']


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Hospital_code                      318438 non-null  int64  
 1   Hospital_type_code                 318438 non-null  int64  
 2   City_Code_Hospital                 318438 non-null  int64  
 3   Hospital_region_code               318438 non-null  int64  
 4   Available Extra Rooms in Hospital  318438 non-null  int64  
 5   Department                         318438 non-null  int64  
 6   Ward_Type                          318438 non-null  int64  
 7   Ward_Facility_Code                 318438 non-null  int64  
 8   Bed Grade                          318325 non-null  float64
 9   patientid                          318438 non-null  int64  
 10  City_Code_Patient                  313906 non-null  float64
 11  Type of Admission                  3184

In [ ]:
#######XGBoost###
# intializing X and Y
import xgboost as XGB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
X=train.drop(['Stay'],axis=1)
y=train['Stay']

# train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

# XGB Classifier
clf=XGB.XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,)
clf.fit(X_train,y_train)

# Predict the values
y_predict=clf.predict(X_test)
test['Stay']=clf.predict(test)

test['Stay1']=test['Stay']

test['Stay']=test['Stay'].map({0:'0-10',1:'11-20',2:'21-30',3:'31-40',4:'41-50',5:'51-60',6:'61-70',7:'71-80',8:'81-90',9:'91-100',10:'More than 100 Days'})

# Accuracy Score
acc=accuracy_score(y_test,y_predict)
print(acc)

0.42670518779047856


In [ ]:
test['Stay'].to_csv('output.csv')